<a href="https://colab.research.google.com/github/ucfilho/AI/blob/main/AI_2023/CV_CNN/class_004_tunekeras_CNN_blood_cells_may_10_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [2]:
# !pip install optuna

In [3]:
from tensorboard.plugins.hparams import api as hp

In [4]:
#import optuna

In [5]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, BatchNormalization
from tensorflow.keras.layers import  Dropout,ZeroPadding2D, MaxPooling2D, Activation, add
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

import cv2
import numpy as np

from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

from glob import glob

import zipfile
import pandas as pd


from sklearn.metrics import classification_report, confusion_matrix

In [6]:
IMAGE_SIZE = [224,224]
ImgSize = IMAGE_SIZE[0]
epochs = 16
batch_size = 128

In [7]:
!gdown 1CU4UkqJae07u1tKfp2rajWwwZA0SKiIm

Downloading...
From: https://drive.google.com/uc?id=1CU4UkqJae07u1tKfp2rajWwwZA0SKiIm
To: /content/blood_cell_images.zip
100% 110M/110M [00:01<00:00, 57.5MB/s] 


In [8]:
FILE='blood_cell_images.zip'

In [9]:
archive = zipfile.ZipFile(FILE, 'r')
archive.extractall()

In [10]:
PARAMS = {'lr':0.005, 
          'epochs':10,
          'batch_size':32}


In [11]:
!ls

blood_cell_images  blood_cell_images.zip  __MACOSX  sample_data


In [12]:
train_path = 'blood_cell_images/TRAIN'
test_path = 'blood_cell_images/TEST'
images_test = glob(test_path + '/*/*.jp*g')
images_train = glob(train_path + '/*/*.jp*g')

In [13]:
folders_train = glob(train_path + '/*')
folders_test = glob(test_path + '/*')

In [14]:
folders_train

['blood_cell_images/TRAIN/LYMPHOCYTE',
 'blood_cell_images/TRAIN/MONOCYTE',
 'blood_cell_images/TRAIN/EOSINOPHIL',
 'blood_cell_images/TRAIN/NEUTROPHIL']

In [15]:
folders_test

['blood_cell_images/TEST/LYMPHOCYTE',
 'blood_cell_images/TEST/MONOCYTE',
 'blood_cell_images/TEST/EOSINOPHIL',
 'blood_cell_images/TEST/NEUTROPHIL']

In [16]:
ntrain, ntest = len(images_train),len(images_test)
ntrain,ntest

(9957, 2487)

In [17]:
images_train[:4]
num_classes = 4

In [18]:
types = ['NEUTROPHIL', 'EOSINOPHIL', 'MONOCYTE', 'LYMPHOCYTE']

In [19]:
size = 20 # resize the images considering this value
n_epochs = 1000

In [20]:

def load_images(images,types=types, dim = (size,size)):

  nsize = len(images)
  X =[]
  y =[]
  names =[]
  k = 0

  for i in range(nsize): 
    img = image.load_img(images[i])
    im = cv2.resize(np.array(img), dim)
    im = preprocess_input(im)
    k += 1
    if( k % 3 == 0):
      X.append(im)

      if(types[0] in images[i]):
        names.append(types[0])
        y.append(0)
      elif(types[1] in images[i]):
        names.append(types[1])
        y.append(1)
      elif(types[2] in images[i]):
        names.append(types[2])
        y.append(2)
      else:
        names.append(types[3])
        y.append(3)

  X =np.array(X)
  return X,y,names

In [21]:

X_train, y_train, names_train = load_images(images_train)
X_test, y_test, names_test = load_images(images_test)

In [22]:
X_train[0].shape

(20, 20, 3)

In [23]:
df = pd.DataFrame({'name':names_train,'class':y_train})
y_train = pd.get_dummies(df['class'])
y_train

,0,1,2,3
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
3314,1,0,0,0
3315,1,0,0,0
3316,1,0,0,0
3317,1,0,0,0


In [24]:
for i in range(4): 
  df_plot= df[(df["class"] == i)]
  n= df_plot.shape[0]
  print(types[i],n)

NEUTROPHIL 833
EOSINOPHIL 833
MONOCYTE 826
LYMPHOCYTE 827


In [25]:
df = pd.DataFrame({'name':names_test,'class':y_test})
y_test = pd.get_dummies(df['class'])
y_test

,0,1,2,3
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
824,1,0,0,0
825,1,0,0,0
826,1,0,0,0
827,1,0,0,0


In [26]:

HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.2, 0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd','RMSprop']))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([ 1e-3, 1e-4]))
HP_EPOCHS = hp.HParam('epochs', hp.Discrete([100, 200]))
METRIC_ACCURACY = 'accuracy'

In [27]:
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[ HP_DROPOUT,HP_LEARNING_RATE, HP_OPTIMIZER, HP_EPOCHS],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [28]:
def train_test_model(hparams):
  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same',
                  input_shape=X_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(hparams[HP_DROPOUT]))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(hparams[HP_DROPOUT]))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  if 'adam'== hparams[HP_OPTIMIZER]:
    optimizer = optimizer=tf.keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING_RATE])
  elif 'sgd'== hparams[HP_OPTIMIZER]:
    optimizer = optimizer=tf.keras.optimizers.SGD(learning_rate=hparams[HP_LEARNING_RATE])
  else: # 'RMSprop'
    optimizer = optimizer=tf.keras.optimizers.RMSprop(learning_rate=hparams[HP_LEARNING_RATE]/10)

  #optimizer=hparams[HP_OPTIMIZER]
  #optimizer=hparams[HP_OPTIMIZER](learning_rate = hparams[HP_LEARNING_RATE])
  #optimizer= tf.keras.optimizers.RMSprop(learning_rate=hp_learning_rate)

  model.compile(optimizer=optimizer,
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])  
  model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_test, y_test),
    epochs= hparams[HP_EPOCHS],
    verbose=0) # verbose=2
  
  _, accuracy = model.evaluate(X_test)
  return accuracy

In [29]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [32]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [ ]:
session_num = 0
for learning_rate in HP_LEARNING_RATE.domain.values:
  for optimizer in HP_OPTIMIZER.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
      for epochs in HP_EPOCHS.domain.values:

        hparams = {
            HP_LEARNING_RATE: learning_rate,
            HP_DROPOUT: dropout_rate,
            HP_EPOCHS: epochs,
            HP_OPTIMIZER: optimizer
            
        }

        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)
        session_num += 1

--- Starting trial: run-0
{'learning_rate': 0.0001, 'dropout': 0.2, 'epochs': 100, 'optimizer': 'RMSprop'}
26/26 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
--- Starting trial: run-1
{'learning_rate': 0.0001, 'dropout': 0.2, 'epochs': 200, 'optimizer': 'RMSprop'}
26/26 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
--- Starting trial: run-2
{'learning_rate': 0.0001, 'dropout': 0.5, 'epochs': 100, 'optimizer': 'RMSprop'}
26/26 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
--- Starting trial: run-3
{'learning_rate': 0.0001, 'dropout': 0.5, 'epochs': 200, 'optimizer': 'RMSprop'}
26/26 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
--- Starting trial: run-4
{'learning_rate': 0.0001, 'dropout': 0.2, 'epochs': 100, 'optimizer': 'adam'}
26/26 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.00